# 03. Transfer Learning (ResNet50)

## Introduction
This notebook implements Transfer Learning using a pre-trained ResNet50 model.
It is fully self-contained.

## Setup

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

tf.random.set_seed(42)
np.random.seed(42)

2025-11-24 10:10:49.468598: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-24 10:10:49.636615: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/mathieu/Works/Projet-PIC/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/mathieu/Works/Projet-PIC/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major

## 1. Data Loading
Using the same strategy as the baseline: training on the multi-class data from the test folder for demonstration.

In [3]:
import sys
sys.path.append('..')
from src.preprocessing.dataset import load_and_split_data
from tensorflow.keras.applications.resnet50 import preprocess_input

IMG_SIZE = (256, 256)
BATCH_SIZE = 32
DATA_DIR = "../data/raw"
TARGET_CATEGORY = 'capsule' # Train only on this category

# Load data
print(f"Loading and splitting data for {TARGET_CATEGORY}...")
train_df, val_df, test_df, class_names = load_and_split_data(DATA_DIR, target_category=TARGET_CATEGORY, augment=True)
num_classes = len(class_names)

print(f"Classes: {class_names}")
print(f"Number of classes: {num_classes}")

# Dataset creation helper
def process_path(filepath, label):
    img = tf.io.read_file(filepath)
    img = tf.io.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    return img, label

def create_dataset(dataframe, batch_size=32, shuffle=False):
    filepaths = dataframe['filepath'].values
    labels = dataframe['label'].values
    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    ds = ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

AUTOTUNE = tf.data.AUTOTUNE

# Create base datasets
train_ds = create_dataset(train_df, BATCH_SIZE, shuffle=True)
val_ds = create_dataset(val_df, BATCH_SIZE, shuffle=False)
test_ds = create_dataset(test_df, BATCH_SIZE, shuffle=False)

# Apply ResNet preprocessing
def preprocess_resnet(image, label):
    return preprocess_input(image), label

train_ds = train_ds.map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(preprocess_resnet, num_parallel_calls=AUTOTUNE)

print("Datasets created and preprocessed.")

Loading and splitting data for capsule...
Classes: ['capsule_crack', 'capsule_faulty_imprint', 'capsule_good', 'capsule_poke', 'capsule_scratch', 'capsule_squeeze']
Number of classes: 6
Datasets created and preprocessed.


## 2. Model Definition
We load ResNet50 (ImageNet weights), freeze the base, and add a custom classification head.

In [ ]:
def create_resnet_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze base model
    base_model.trainable = False
    
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs, name="resnet50_transfer")
    return model

model = create_resnet_model(IMG_SIZE + (3,), num_classes)
model.summary()

## 3. Training
Training the top layers.

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
)

## 4. Fine-tuning (Optional)
Unfreezing the last few layers of ResNet for better performance.

In [ ]:
base_model = model.layers[1]
base_model.trainable = True

# Freeze all except last 20 layers
for layer in base_model.layers[:-20]:
    layer.trainable = False
    
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # Lower LR
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
)

In [ ]:
# Advanced Evaluation
from src.evaluation.benchmark import evaluate_model

print("Evaluating model with advanced metrics...")
results = evaluate_model(model, test_ds)

print("\nEvaluation Results:")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")